In [1]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from sklearn.utils import resample
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
from optimum.bettertransformer import BetterTransformer
import torch

c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.1
Is CUDA enabled? True


In [3]:
dataset = load_dataset("financial_phrasebank", 'sentences_50agree')

neutrals = dataset.filter(lambda example: example["label"] == 1)
positives = dataset.filter(lambda example: example["label"] == 2)
negatives = dataset.filter(lambda example: example["label"] == 0)

short_positives = positives['train'].select(range(len(negatives['train'])))

print(short_positives)

downsized_dataset = concatenate_datasets([short_positives, negatives['train']])

print(downsized_dataset)

Found cached dataset financial_phrasebank (C:/Users/Gusso/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
100%|██████████| 1/1 [00:00<00:00, 71.44it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-4069f8bdc2072a81.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-255302c37e6b94c9.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-913aca77f5bc0114.arrow


Dataset({
    features: ['sentence', 'label'],
    num_rows: 604
})
Dataset({
    features: ['sentence', 'label'],
    num_rows: 1208
})


In [4]:
def replace_values(example):
    # Replace values in the label column
    if "label" in example:
        example["label"] = 1 if example["label"] == 2 else example["label"]
    return example

# Use the map function to apply the custom mapping
downsized_dataset = downsized_dataset.map(replace_values)
downsized_dataset = downsized_dataset.train_test_split(test_size=0.1, shuffle=True)
print(downsized_dataset)

Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-9ab259d81c7af6d1.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1087
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 121
    })
})


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding='max_length', truncation=True)

tokenized_datasets = downsized_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_eval = downsized_dataset['test'].map(tokenize_function, batched=True)

c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Gusso\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

model = model.to(0)

print(model.get_memory_footprint())

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


437943304


In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [8]:
trainer.train()

                                                 
 33%|███▎      | 136/408 [00:35<01:03,  4.27it/s]

{'eval_loss': 0.14862680435180664, 'eval_accuracy': 0.9669421487603306, 'eval_runtime': 1.3065, 'eval_samples_per_second': 92.613, 'eval_steps_per_second': 12.246, 'epoch': 1.0}


                                                 
 67%|██████▋   | 272/408 [01:10<00:31,  4.31it/s]

{'eval_loss': 0.022819897159934044, 'eval_accuracy': 0.9834710743801653, 'eval_runtime': 1.303, 'eval_samples_per_second': 92.863, 'eval_steps_per_second': 12.279, 'epoch': 2.0}


                                                 
100%|██████████| 408/408 [01:46<00:00,  3.84it/s]

{'eval_loss': 0.05054190009832382, 'eval_accuracy': 0.9917355371900827, 'eval_runtime': 1.305, 'eval_samples_per_second': 92.72, 'eval_steps_per_second': 12.261, 'epoch': 3.0}
{'train_runtime': 106.1818, 'train_samples_per_second': 30.711, 'train_steps_per_second': 3.842, 'train_loss': 0.15528183357388364, 'epoch': 3.0}


TrainOutput(global_step=408, training_loss=0.15528183357388364, metrics={'train_runtime': 106.1818, 'train_samples_per_second': 30.711, 'train_steps_per_second': 3.842, 'train_loss': 0.15528183357388364, 'epoch': 3.0})

In [9]:
model = BetterTransformer.transform(model)

trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [10]:
validation_dataset = load_dataset('zeroshot/twitter-financial-news-sentiment')

def filter_function(example):
    return example["label"] != 2

# Use the filter method to remove records
filtered_dataset = validation_dataset.filter(filter_function)

def tokenize_function2(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

tokenized_datasets_validation = filtered_dataset.map(tokenize_function2, batched=True)

# Print the modified dataset
print(dataset)

Found cached dataset csv (C:/Users/Gusso/.cache/huggingface/datasets/zeroshot___csv/zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 90.91it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b2d98c45395bc49c.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-98a6e9e55fa382e5.arrow
                                                                 

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4846
    })
})


In [11]:
trainer2.evaluate(eval_dataset=tokenized_datasets_validation['validation'])

c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\optimum\bettertransformer\models\encoder_models.py:301: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\NestedTensorImpl.cpp:180.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, ~attention_mask)
100%|██████████| 103/103 [00:02<00:00, 43.31it/s]


{'eval_loss': 0.8126084208488464,
 'eval_accuracy': 0.8369829683698297,
 'eval_runtime': 2.6235,
 'eval_samples_per_second': 313.32,
 'eval_steps_per_second': 39.26}